In [1]:
## This script compares Spanglish_test_conll_unlabeled 
## with tweets found in dev, train, and sp_train
## to see if the labeled data exists

In [6]:
train_con_fi = '../data/Semeval_2020_task9_data/Spanglish/train.conll'
sp_train_con_fi = '../data/Semeval_2020_task9_data/Spanglish/Spanglish_train.conll'
sp_test_fi = '../data/Semeval_2020_task9_data/Spanglish/Spanglish_test_conll_unlabeled.txt'
sp_dev_fi = '../data/Semeval_2020_task9_data/Spanglish/Spanglish_dev.conll'

In [7]:
def structure_data(fi):
    print('Now on {}'.format(fi.split('//')[-1]))
    with open(fi, 'r', encoding='utf-8') as f:
        lines = [x.replace('\n', '') for x in f.readlines()]
        
    d = {}
    cur_tweet = []
    cur_num = '1'  # all files start with 1
    cur_sentiment = 'positive'   # all files start positive
    for i in range(1, len(lines)):
        l = lines[i]
        data = l.split('\t')
        if l == '' or l == '\t':
            
            tweet_tokes = [x[0].lower() for x in cur_tweet]
            tweet_langs = [x[1] for x in cur_tweet]
            
            # unlabeled doesn't have sentiments
            if 'unlabeled' in fi:
                d[cur_num] = {'tweet_tokens':tweet_tokes, 'tweet_langs':tweet_langs}
            else:
                d[cur_num] = {'tweet_tokens':tweet_tokes, 'tweet_langs':tweet_langs,'sentiment':cur_sentiment}
        elif data[0].startswith('# sent_enum'):
            cur_num = data[0].split('=')[-1].strip()
            cur_sentiment = data[1]
            cur_tweet = []
        
        # other format
        elif data[0] == 'meta':
            if 'unlabeled' in fi:
                cur_num = data[1]
                cur_tweet = []
            else:
                cur_num = data[0].split('=')[-1].strip()
                cur_sentiment = data[1]
                cur_tweet = []
        
        else:
            cur_tweet.append(data)
    return d


In [8]:
train_con_d = structure_data(train_con_fi)
sp_train_con_d = structure_data(sp_train_con_fi)
sp_test_d = structure_data(sp_test_fi)
sp_dev_d = structure_data(sp_dev_fi)

Now on ../../data/Semeval_2020_task9_data/Spanglish/train.conll
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_train.conll
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_test_conll_unlabeled.txt
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_dev.conll


In [9]:
print(len(train_con_d))
train_con_d['1']

12191


{'tweet_tokens': ['after',
  'this',
  'i',
  "'m",
  'just',
  'gonna',
  'go',
  'home',
  'drink',
  'summ',
  'hot',
  'chocolate',
  'con',
  'bolillo',
  'and',
  'sleep'],
 'tweet_langs': ['lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'lang1',
  'ambiguous',
  'lang2',
  'lang2',
  'lang1',
  'lang1'],
 'sentiment': 'positive'}

## Compare train_conll to sp_test

In [10]:
# uses heuristics: 
# if two tweets have identical first 5 words and the same length exactly, they are assumed to be same tweet
# if two tweets have identical last 3 words and the same length exactly, also assumed identical
# this helps with the shitty encoding

In [11]:
com = {}
count = 0
for k in sp_test_d:
    test_tokes = sp_test_d[k]['tweet_tokens']
    found = False
    for k2 in sp_train_con_d:
        sp_train_con_tokes = sp_train_con_d[k2]['tweet_tokens']
        if test_tokes[:5] == sp_train_con_tokes[:5] and len(test_tokes) == len(sp_train_con_tokes):
            com[count+1] = {}
            com[count+1][('test',k)] = sp_test_d[k]
            com[count+1][('sp_train',k2)] = sp_train_con_d[k2]
            count+=1
            found = True
            break
        elif test_tokes[-3:] == sp_train_con_tokes[-3:] and len(test_tokes) == len(sp_train_con_tokes):
            com[count+1] = {}
            com[count+1][('test',k)] = sp_test_d[k]
            com[count+1][('sp_train', k2)] = sp_train_con_d[k2]
            count+=1
            found = True
            break
    if found == False:
        for k2 in train_con_d:
            train_con_tokes = train_con_d[k2]['tweet_tokens']
            if test_tokes[:5] == train_con_tokes[:5]  and len(test_tokes) == len(train_con_tokes):
                com[count+1] = {}
                com[count+1][('test',k)] = sp_test_d[k]
                com[count+1][('train',k2)] = train_con_d[k2]
                count+=1
                found = True
                break
            elif test_tokes[-3:] == train_con_tokes[-3:] and len(test_tokes) == len(train_con_tokes):
                com[count+1] = {}
                com[count+1][('test',k)] = sp_test_d[k]
                com[count+1][('train', k2)] = train_con_d[k2]
                count+=1
                found = True
                break
    if found == False:
        for k2 in sp_dev_d:
            dev_tokes = sp_dev_d[k2]['tweet_tokens']
            if test_tokes[:5] == dev_tokes[:5]  and len(test_tokes) == len(dev_tokes):
                com[count+1] = {}
                com[count+1][('test',k)] = sp_test_d[k]
                com[count+1][('dev', k2)] = sp_dev_d[k2]
                count+=1
                found = True
                break
            elif test_tokes[-3:] == dev_tokes[-3:] and len(test_tokes) == len(dev_tokes):
                com[count+1] = {}
                com[count+1][('test',k)] = sp_test_d[k]
                com[count+1][('dev', k2)] = sp_dev_d[k2]
                count+=1
                found = True
                break
        

In [12]:
# only using
test_tweets = len(sp_test_d)
print('Number of tweets in test: {}'.format(test_tweets))

Number of tweets in test: 3785


In [13]:
# not checking length of tweets in 5 identical ones
test_founds = len(com)
print('Number of tweets in comparison: {}'.format(test_founds))

Number of tweets in comparison: 2200


In [9]:
# encoding differences are from the missing 1500 tweets?

In [14]:
com[1000].keys()

dict_keys([('test', '1574'), ('train', '3716')])

In [15]:
pos = 0
neg = 0
neu = 0
tups = []

for k in com:
    for source in com[k]:
        if source[0]!= 'test':
            val = com[k][source]['sentiment']
            
            tups.append('{}\t{}\t{}'.format('meta',k,val))
            tts = com[k][source]['tweet_tokens']
            tls = com[k][source]['tweet_langs']
            for i in range(len(tts)):
                tups.append('{}\t{}'.format(tts[i],tls[i]))
            tups.append('')
            if val.startswith('pos'):
                pos+=1
            elif val.startswith('neg'):
                neg+=1
            else:
                neu+=1
    
print('Positive tweets: {}\n\trepresent {} percent of test set'.format(pos,pos/len(com)))
print('Negative tweets: {}\n\trepresent {} percent of test set'.format(neg,neg/len(com)))
print('Neutral tweets: {}\n\trepresent {} percent of test set'.format(neu,neu/len(com)))

Positive tweets: 1854
	represent 0.8427272727272728 percent of test set
Negative tweets: 233
	represent 0.10590909090909091 percent of test set
Neutral tweets: 113
	represent 0.05136363636363636 percent of test set


In [45]:
# possible encoding fix ??

In [46]:
# to restore proper emoji encoding
# https://stackoverflow.com/questions/20108312/how-can-i-restore-proper-encoding-of-4-byte-emoji-characters-that-have-been-stor

In [47]:
out = '../data/Semeval_2020_task9_data/Spanglish/Spanglish_test_conll_labeled.txt'
with open(out, 'w',encoding='utf-8') as fi:
    for t in tups:
        fi.write(t+'\n')